In [83]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


from google.colab import drive
drive.mount('/content/drive',force_remount=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Mounted at /content/drive


In [2]:
data1 = pd.read_csv('/content/drive/My Drive/thesis/feature_engineering/features_b.tsv', sep='\t', engine="python")
train = data1.filter(['q1','q2','annotation'], axis=1)
train = train.rename({'q1': 'question1', 'q2': 'question2' }, axis=1)

In [3]:
def preprocess(series):
  #remove characters other than alphabets & numerics
  words = re.sub("[^A-Za-z0-9]"," ",series).lower().split()

  #lemmatize words
  lemm = WordNetLemmatizer()
  stpwords = stopwords.words('english')
  lemmitized = [lemm.lemmatize(word) for word in words if word not in stpwords]
  sent = ' '.join(lemmitized)
  return sent

In [4]:
train['question1'] =train['question1'].apply(preprocess)#Apply preprocessing
train['question2'] =train['question2'].apply(preprocess)

In [5]:
def concat(ser):#concatenate Question 1 & Question 2
  print(ser['question1'])
  return 1
train['combine'] = train.apply(lambda ser: ser['question1'] + " " + ser['question2'],axis=1)
train.head(10)

,question1,question2,annotation,combine
0,champion 120v generator want use plug preinsta...,120 240 dryer canada want use 240v country,1,champion 120v generator want use plug preinsta...
1,champion 120v generator want use plug preinsta...,powering 220v neutral appliance 120v split phase,1,champion 120v generator want use plug preinsta...
2,champion 120v generator want use plug preinsta...,possible wire 240v outlet need 240v anymore tw...,1,champion 120v generator want use plug preinsta...
3,run conduit containing 120v wire inside larger...,advantage using emt plastic flexible conduit l...,0,run conduit containing 120v wire inside larger...
4,run conduit containing 120v wire inside larger...,cable conduit use run new 240 volt line oven,1,run conduit containing 120v wire inside larger...
5,run conduit containing 120v wire inside larger...,conduit size sub panel run,1,run conduit containing 120v wire inside larger...
6,run conduit containing 120v wire inside larger...,run thhn separate washer dryer circuit conduit,1,run conduit containing 120v wire inside larger...
7,run conduit containing 120v wire inside larger...,8 thhn conduit 50 amp circuit,1,run conduit containing 120v wire inside larger...
8,run conduit containing 120v wire inside larger...,derating wire conduit round,1,run conduit containing 120v wire inside larger...
9,run conduit containing 120v wire inside larger...,new manufactured home permanent foundation nee...,0,run conduit containing 120v wire inside larger...


In [6]:
cv = TfidfVectorizer(max_features=50000)#Word to Vectors using Tf-Idf

#Take combine questions data as X
X = cv.fit_transform(train['combine'])
y = np.array(train['annotation'])
print(X.shape)

#Tarin-Test Spilt
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05)
print(X_train.shape,X_test.shape)

(8299, 4215)
(7884, 4215) (415, 4215)


In [7]:
naive_model = MultinomialNB()#Training
naive_model.fit(X_train,y_train)

#Predictions
y_pred_train = naive_model.predict(X_train)
y_pred_test = naive_model.predict(X_test)

In [8]:
accuracy_train = sum((y_pred_train == y_train).astype(int))/len(y_train)
accuracy_test = sum((y_pred_test == y_test).astype(int))/len(y_test)
print(accuracy_train,accuracy_test)

0.5702688990360223 0.5012048192771085


In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
!pip install transformers==2.11.0
import transformers

     |████████████████████████████████| 674 kB 14.4 MB/s 
     |████████████████████████████████| 1.2 MB 63.9 MB/s 
     |████████████████████████████████| 5.6 MB 25.2 MB/s 
     |████████████████████████████████| 895 kB 60.3 MB/s 


In [29]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2

# Labels in our dataset.
labels = [0,1,2]

data1 = pd.read_csv('/content/drive/My Drive/thesis/feature_engineering/features_b.tsv', sep='\t', engine="python")
df = data1.filter(['q1','q2','annotation'], axis=1)
df = df.rename({'q1': 'question1', 'q2': 'question2' }, axis=1)

In [11]:
mask = np.random.rand(len(df)) < 0.7
train_df = df[mask]
not_train = df[~mask]

#create mask for val-test distribution
mask = np.random.rand(len(not_train)) < 0.5
test_df = not_train[mask]
val_df = not_train[~mask]
val_df.head(5)

,question1,question2,annotation
0,"""I have a Champion 120V generator, I want to u...","""I have a 120/240 dryer from Canada, and want ...",1
14,30V on 120 circuit - swapped dishwashers and c...,"""AC power leaving the circuit breaker, but no ...",0
22,How much 120 from 240 is too much?,Can I safely create another 120V outlet using ...,1
30,Is it possible to wire a 240v outlet so that a...,What are my options for installing 240v recept...,1
35,Is it possible to wire a 240v outlet so that a...,Installing a 240V 30 A outlet for EV charging,1


In [12]:
print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {val_df.shape[0]}")
print(f"Total test samples: {test_df.shape[0]}")

Total train samples : 5714
Total validation samples: 1289
Total test samples: 1296


In [13]:
print("Train Target Distribution")
print(train_df.annotation.value_counts())

print("Validation Target Distribution")
print(val_df.annotation.value_counts())

Train Target Distribution
1    2465
0    1714
2    1535
Name: annotation, dtype: int64
Validation Target Distribution
1    572
0    392
2    325
Name: annotation, dtype: int64


In [14]:
y_train = tf.keras.utils.to_categorical(train_df.annotation, num_classes=3)#One hot encodding representation
print(f"y_train.shape:{y_train.shape}")

y_val = tf.keras.utils.to_categorical(val_df.annotation, num_classes=3)
print(f"y_val.shape:{y_val.shape}")

y_test = tf.keras.utils.to_categorical(test_df.annotation, num_classes=3)
print(f"y_test.shape:{y_test.shape}")

y_train.shape:(5714, 3)
y_val.shape:(1289, 3)
y_test.shape:(1296, 3)


In [15]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [16]:
strategy = tf.distribute.MirroredStrategy()# Create the model under a distribution strategy scope.

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    sequence_output, pooled_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(sequence_output)
    
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f63ed62abd0>
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                           

In [17]:
train_data = BertSemanticDataGenerator(
    train_df[["question1", "question2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
val_data = BertSemanticDataGenerator(
    val_df[["question1", "question2"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [18]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
     use_multiprocessing=True,
    workers=-1,
)

Epoch 1/2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
178/178 [==============================] - 83s 372ms/step - loss: 1.0483 - acc: 0.4445 - val_loss: 1.0278 - val_acc: 0.4680
Epoch 2/2
178/178 [=======

In [19]:
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    ((None, 128, 768),   109482240   ['input_ids[0][0]',              
                                 (None, 768))                     'attention_masks[0][0]',    

In [20]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Epoch 1/2
178/178 [==============================] - 176s 878ms/step - loss: 0.9544 - accuracy: 0.5174 - val_loss: 0.9823 - val_accuracy: 0.4836
Epoch 2/2
178/178 [==============================] - 157s 881ms/step - loss: 0.8954 - accuracy: 0.5671 - val_loss: 0.9806 - val_accuracy: 0.4906


In [21]:
test_data = BertSemanticDataGenerator(
    test_df[["question1", "question2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

40/40 [==============================] - 12s 294ms/step - loss: 0.9546 - accuracy: 0.5195


[0.9546359181404114, 0.51953125]

In [55]:
def check_similarity(sentence1, sentence2):
  sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
  test_data = BertSemanticDataGenerator(
      sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
  )

  proba = model.predict(test_data[0])[0]
  idx = np.argmax(proba)
  proba = f"{proba[idx]: .2f}"
  pred = labels[idx]
  return pred, proba

In [74]:
test_df[test_df["annotation"] == 2].shape[0]

353

In [81]:
ind = np.random.randint(0,353)
#Similar Questions
q1 = test_df[test_df["annotation"] == 2].iloc[ind]['question1']
q2 = test_df[test_df["annotation"] == 2].iloc[ind]['question2']
print(q1+"\n"+q2)
check_similarity(q1,q2)

Confusion over 220 and 230 volt outlets
Receving 120 and 220 volt readings when testing a home outlet


(2, ' 0.59')

In [76]:
test_df[test_df["annotation"] == 1].shape[0]

569

In [82]:
ind = np.random.randint(0,569) 
#relavant Questions
q1 = test_df[test_df["annotation"] == 1].iloc[ind]['question1']
q2 = test_df[test_df["annotation"] == 1].iloc[ind]['question2']
print(q1+"\n"+q2)
check_similarity(q1,q2)


Remove hard amber glue from tile and bathtub?
How to remove glue from ceiling


(1, ' 0.38')

In [78]:
test_df[test_df["annotation"] == 0].shape[0]

374

In [79]:
ind = np.random.randint(0,374)
#No-Relation Questions
q1 = test_df[test_df["annotation"] == 0].iloc[ind]['question1']
q2 = test_df[test_df["annotation"] == 0].iloc[ind]['question2']
print(q1+"\n"+q2)
check_similarity(q1,q2)

Dryer 240 to 120 question
How to Use GFCI Square Cover?


(0, ' 0.69')

In [99]:
preds = model.predict(test_data[0])
print(preds)
#print(classification_report(np.argmax(y_test,axis=1),np.argmax(preds,axis=1)))

[[0.13353136 0.527558   0.33891067]]
